In [1]:
import pandas as pd

#All Functons Used

def combine(dataSet, combine):
    #Sets all crimes as non forcible
    dataSet.loc[dataSet.Forcible == 'Y', 'Forcible']= 'N'
    
    #If a crime has a negative indicator then remove
    dataSet.loc[(dataSet.Offense.str.contains('NO ASSAULT') == True), 'Offense']= 'N'
    
    #Reported as Section can sometimes differ from the offense Rape is included in sexual
    dataSet.loc[(dataSet.Reported_As.str.contains('SEXUAL') == True), 'Offense'] = 'SEXUAL'
    dataSet.loc[(dataSet.Offense.str.contains('RAPE') == True), 'Offense']= 'SEXUAL'
    
    #Both Homicide and manslaughter are included as murder
    dataSet.loc[(dataSet.Offense.str.contains('HOMICIDE') == True), 'Offense']= 'MURDER'
    dataSet.loc[(dataSet.Offense.str.contains('MANSLAUGHTER') == True), 'Offense']= 'MURDER'
    
    #Combines all offenses with the same crime label as such
    #Also sets those crimes to forcible 
    for x in combine:
        dataSet.loc[(dataSet.Offense.str.contains('NO '+ x) == True), 'Offense']= 'N'
        dataSet.loc[(dataSet.Offense.str.contains(x) == True), 'Offense']= x
        dataSet.loc[dataSet.Offense == x, 'Forcible']= 'Y'
    
    #Removes all non forcible crimes and crimes without victims
    dataSet = dataSet[dataSet.Forcible == 'Y']
    dataSet = dataSet[dataSet.Victim_Gender.notna()]
    
    return dataSet

def breakdown(dataSet, crime_labels):
    
    #Prints out each crime breakdown
    for x in crime_labels:
        sum = len(dataSet[dataSet.Offense == x])
        print(x + (10 - len(x)) * " " + str(sum))


In [11]:
raw_data = pd.read_csv('Data/ch_crime.csv')

#Take out unnecessary variables
formated_data = raw_data[['X','Y', 'Offense', 'Date_of_Occurrence', 'Forcible', 'Weapon_Description', 'Victim_Gender', 'Reported_As']]
formated_data = formated_data[formated_data.Victim_Gender.notna()]

formated_data.loc[formated_data.Victim_Gender == formated_data['Victim_Gender'][0], 'Gender'] = 1
formated_data.loc[formated_data.Gender != formated_data.Gender.notna(), 'Gender'] = 0

#Rename X and Y
formated_data = formated_data.rename(columns = {'X':'Longitude','Y':'Latitude'})

In [3]:
# split column and add new columns to formated_data

formated_data[['Date', 'Time']] = formated_data['Date_of_Occurrence'].str.split(' ', expand=True)
formated_data['Time'] = formated_data['Time'].str[:-9]
formated_data['Time'] = formated_data['Time'].astype(float)

In [4]:
#Combines like crimes and removed everything else

#crime_labels = ['SEXUAL', 'ASSAULT', 'VANDALISM', 'B&E', 'LACENY',  'ROBBERY', 'MURDER', 'BURGLARY']
crime_labels = ['SEXUAL', 'ASSAULT', 'VANDALISM', 'B&E', 'LARCENY']
combined_data = combine(formated_data, crime_labels)

breakdown(combined_data, crime_labels)

SEXUAL    1017
ASSAULT   3141
VANDALISM 1734
B&E       5931
LARCENY   5145


In [5]:
#Sorts data by Geographic location
combined_data = combined_data.sort_values(['Longitude', 'Latitude'])

#Resets mixed index
cleanSet = combined_data.reset_index()

#Removes all no longer nessassary infomation
cleanSet = cleanSet[['Longitude','Latitude', 'Offense', 'Time', 'Gender']]

cleanSet

,Longitude,Latitude,Offense,Time,Gender
0,-79.452193,35.737344,ASSAULT,2.0,1.0
1,-79.185550,35.922087,B&E,14.0,0.0
2,-79.185550,35.922087,B&E,14.0,0.0
3,-79.185550,35.922087,B&E,14.0,1.0
4,-79.185501,35.922094,VANDALISM,11.0,1.0
...,...,...,...,...,...
16963,-78.974294,35.915845,B&E,11.0,1.0
16964,-78.962904,35.967732,LARCENY,0.0,0.0
16965,-78.958768,35.919429,LARCENY,20.0,0.0
16966,-78.935082,35.979530,B&E,3.0,1.0
